This notebook replicates the steps shown in the free course procided by SpatialThoughts

https://courses.spatialthoughts.com/python-foundation.html#the-python-standard-library

In [3]:
from geopy import distance

In [4]:
nouadhibou = (20.9346, -17.0348)
richat_structure = (20.4879, -11.7559)


# Calculating distance

## distance.great_circle()
 Calculates the distance on a great circle using haversine formula


In [5]:
straight_line_distance = distance.great_circle(nouadhibou, richat_structure)
print(straight_line_distance)

551.2694192780905 km


## distance.geodesic()

Calculates the distance using a chosen ellipsoid using vincenty’s formula

In [6]:
ellipsoid_distance = distance.geodesic(nouadhibou, richat_structure)
print(ellipsoid_distance)

552.0911860567066 km


In [7]:
ellipsoid_distance_wgs84 = distance.geodesic(nouadhibou, richat_structure, ellipsoid='WGS-84')
print(ellipsoid_distance_wgs84)


552.0911860567066 km


default ellipsoid is wgs-84

In [8]:
delta_distances = ellipsoid_distance - straight_line_distance
print(delta_distances)

0.8217667786161655 km


# Using Web APIs

In [9]:
#sample geojson structure string
geojson_string = '''
{
  "type": "FeatureCollection",
  "features": [
    {"type": "Feature",
      "properties": {"name": "San Francisco"},
      "geometry": {"type": "Point", "coordinates": [-121.5687, 37.7739]}
    }
  ]
}
'''
print(geojson_string)


{
  "type": "FeatureCollection",
  "features": [
    {"type": "Feature",
      "properties": {"name": "San Francisco"},
      "geometry": {"type": "Point", "coordinates": [-121.5687, 37.7739]}
    }
  ]
}



In [10]:
import json

In [11]:
data = json.loads(geojson_string)
print(type(data))

<class 'dict'>


In [12]:
print(data)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'San Francisco'}, 'geometry': {'type': 'Point', 'coordinates': [-121.5687, 37.7739]}}]}


In [13]:
city_data=data['features'][0]
print(city_data)


{'type': 'Feature', 'properties': {'name': 'San Francisco'}, 'geometry': {'type': 'Point', 'coordinates': [-121.5687, 37.7739]}}


# The requests module
To query a server, we send a GET request with some parameters and the server sends a response back. The requests module allows you to send HTTP requests and parse the responses using Python.

The response contains the data received from the server. It contains the HTTP status_code which tells us if the request was successful. HTTP code 200 stands for Sucess OK.

In [14]:
import requests

In [15]:
response = requests.get('https://www.spatialthoughts.com')

In [16]:
print(response.status_code)


200


https://openrouteservice.org/dev/#/signup

In [17]:
ORS_API_KEY="replace this with your key"

API limited to a few cities, Ouadane in Mauritania does not work

In [18]:
nouadhibou_coordinates = (20.9346, -17.0348)
nouakchott_coordinates = (18.0864, -15.9751)

parameters = {
    'api_key': ORS_API_KEY,
    'start': '{},{}'.format(nouadhibou_coordinates[1], nouadhibou_coordinates[0]),
    'end': '{},{}'.format(nouakchott_coordinates[1], nouakchott_coordinates[0])
}


In [19]:
response = requests.get(
    'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)


In [20]:
if response.status_code == 200:
    print('Request successful.')
    data = response.json()
else:
    print('Request failed.')

Request failed.


In [21]:
summary = data['features'][0]['properties']['summary']
print(summary)

KeyError: 'summary'

In [ ]:
data = response.json()

In [ ]:
summary=data['features'][0]['properties']['summary']
print(summary)

{'distance': 477696.7, 'duration': 19523.5}


In [ ]:
#converting to km
distance = summary['distance']
print(distance/1000)


477.6967


# Geocoding addresses using geopy

### Sample code with addresses to geocode


In [ ]:
# List of Hurricane Evacuation Centers in New York City with Addresses
# Each item is a tuple with the name of the center and its address
locations = [
    ('Norman Thomas HS (ECF)', '111 E 33rd St, NYC, New York'),
    ('Midtown East Campus', '233 E 56th St, NYC, New York'),
    ('Louis D. Brandeis HS', '145 W 84th St, NYC, New York'),
    ('Martin Luther King, Jr. HS', '122 Amsterdam Avenue, NYC, New York'),
    ('P.S. 48', '4360 Broadway, NYC, New York')
]

In [ ]:
locations

[('Norman Thomas HS (ECF)', '111 E 33rd St, NYC, New York'),
 ('Midtown East Campus', '233 E 56th St, NYC, New York'),
 ('Louis D. Brandeis HS', '145 W 84th St, NYC, New York'),
 ('Martin Luther King, Jr. HS', '122 Amsterdam Avenue, NYC, New York'),
 ('P.S. 48', '4360 Broadway, NYC, New York')]

In [23]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_geocoder")


In [31]:
geocoded_locations=[]


In [32]:
for location in locations:
    name, address = location
    location_info = geolocator.geocode(address)
    if location_info is not None:
        latitude, longitude = location_info.latitude, location_info.longitude
        geocoded_locations.append((name, latitude, longitude))
        print(f'{name}: Latitude={latitude}, Longitude={longitude}')
    else:
        print(f'Geocoding failed for {name}: {address}')

Norman Thomas HS (ECF): Latitude=40.7462177, Longitude=-73.9809816
Midtown East Campus: Latitude=40.65132465, Longitude=-73.92421646290632
Louis D. Brandeis HS: Latitude=40.7857432, Longitude=-73.9742029
Martin Luther King, Jr. HS: Latitude=40.7747751, Longitude=-73.9853689
P.S. 48: Latitude=40.8532731, Longitude=-73.9338592


In [33]:
# Print the geocoded locations or use them as needed
for name, latitude, longitude in geocoded_locations:
    print(f'{name}: Latitude={latitude}, Longitude={longitude}')

Norman Thomas HS (ECF): Latitude=40.7462177, Longitude=-73.9809816
Midtown East Campus: Latitude=40.65132465, Longitude=-73.92421646290632
Louis D. Brandeis HS: Latitude=40.7857432, Longitude=-73.9742029
Martin Luther King, Jr. HS: Latitude=40.7747751, Longitude=-73.9853689
P.S. 48: Latitude=40.8532731, Longitude=-73.9338592


In [34]:
geocoded_locations

[('Norman Thomas HS (ECF)', 40.7462177, -73.9809816),
 ('Midtown East Campus', 40.65132465, -73.92421646290632),
 ('Louis D. Brandeis HS', 40.7857432, -73.9742029),
 ('Martin Luther King, Jr. HS', 40.7747751, -73.9853689),
 ('P.S. 48', 40.8532731, -73.9338592)]

## Part 2
Get a list of 5 addresses in your city and geocode them.

You can use Nominatim geocoder. Nominatim is based on OpenStreetMap and the it’s geocoding quality varies from country to country. You can visit https://openstreetmap.org/ and search for your address. It uses Nominatim geocoder so you can check if your address is suitable for this service.

Many countries of the world do not have structured addresses and use informal or landmark based addresses. There are usually very difficult to geocode accurately. If you are trying to geocode such addresses, your best bet is to truncate the address at the street or locality level.



In [44]:


# Generate 5 random addresses in Madrid city center
madrid_addresses = ["Avenida de la Castellana 34, Madrid", "Calle de Alberto Bosch 10, Madrid",
                    "C. de Almadén, 28014 Madrid", "C. de la Alameda, 102, 28014 Madrid"]

In [45]:
madrid_addresses

['Avenida de la Castellana 34, Madrid',
 'Calle de Alberto Bosch 10, Madrid',
 'C. de Almadén, 28014 Madrid',
 'C. de la Alameda, 102, 28014 Madrid']

In [46]:
# Create a list to store the geocoded locations
madrid_locations = []

# Loop through the addresses and geocode each one
for address in madrid_addresses:
    location_info = geolocator.geocode(address)
    if location_info is not None:
        latitude, longitude = location_info.latitude, location_info.longitude
        madrid_locations.append((address, latitude, longitude))
    else:
        print(f'Geocoding failed for address: {address}')

# Print the geocoded locations or use them as needed
for address, latitude, longitude in madrid_locations:
    print(f'Address: {address}')
    print(f'Latitude: {latitude}, Longitude: {longitude}')

Geocoding failed for address: Avenida de la Castellana 34, Madrid
Geocoding failed for address: Calle de Alberto Bosch 10, Madrid
Geocoding failed for address: C. de Almadén, 28014 Madrid
Address: C. de la Alameda, 102, 28014 Madrid
Latitude: 40.41178, Longitude: -3.6938052


In [47]:
madrid_locations

[('C. de la Alameda, 102, 28014 Madrid', 40.41178, -3.6938052)]

Not working wit all addresses

# Geopandas

In [48]:
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'